In [ ]:
include("../../src/Yggdrasil.jl")
using .Yggdrasil

In [ ]:
midpoint = [16.27207,  0.0, 6.1138]
û     = [0.707, 0.0, -0.68]
num_points = 50

n̂     = [0.0, 1.0, 0.0]
half_length = 0.3 # on either side of the line
half_width  = 0.2 # on either side of the line
n_perp = 50;

In [ ]:
#------------------ get points along a line ------------------
# midpoint: Center of the line
# û: Direction vector of the line
# half_length: Half the length of the line segment (on either side of midpoint)
# num_points: Total number of points to generate along the line segment 
line = get_line(midpoint,
                  û,
                  half_length,
                  num_points);

In [ ]:
#----------- get a 2D plane around a line --------------------------------
#  midpoint: Center of the line
#  û: Direction vector of the line
#  n̂: Normal vector to the plane
#  half_length: Half the length of the line segment (on either side of midpoint)
#  half_width: Half the width of the plane (on either side of the line)
#  n_perp: Number of points perpendicular to the line
#  n_par (optional): Number of points parallel to the line
#                   If not provided, it will be calculated based on the ratio of half_length to half_width
#                   and the number of perpendicular points to have roughly equal spacing.
n_par = 50 # Optional, can be set to `nothing` to auto-calculate
line, perp_vals, par_vals, Δpar, Δperp, grid = generate_line_plane(midpoint,
                                                                    û,
                                                                    n̂,
                                                                    half_length,
                                                                    half_width,
                                                                    n_perp; 
                                                                    n_par = n_par);

In [ ]:
#----------- get a 2D plane around a line --------------------------------
#  midpoint: Center of the line
#  û: Direction vector of the line
#  n̂: Normal vector to the plane
#  half_length: Half the length of the line segment (on either side of midpoint)
#  half_width: Half the width of the plane (on either side of the line)
#  n_perp: Number of points perpendicular to the line
#         here n_par is not provided, it will be calculated so Δpar≈Δperp
line, perp_vals, par_vals, Δpar, Δperp, grid = generate_line_plane(midpoint,
                                                                    û,
                                                                    n̂,
                                                                    half_length,
                                                                    half_width,
                                                                    n_perp);

In [ ]:
data_folder = "../example_data/test_01/"    # experiment folder 
snap = 105                                  # snapshot number to load
ref_level = 3                               # load from the second level (refinement level 1)
load_pic = false;                           # if true only PIC patches are loaded, otherwise load MHD patches




Snap_meta = read_snapshot(data_folder, snap)   # Load initial snapshot meta
IDX = Snap_meta.IDX
level_min = Snap_meta.LEVELMIN
level = level_min + ref_level     #


i_start = 101
i_end = 105
stride = 1

#------------ Get area defined by LLC and URC aligned with patches --------------   
    #------------- define midpoint of area and number of patches
    midpoint = [16.27207,  0.0, 6.1138]
    patches = 6
    #-----------------------------------------------------
llc, urc, llc_pos, urc_pos  = get_area(Snap_meta, midpoint, patches, level);

#----------- load data
# 3 deifferent load functions are used to illustrate line analysis
# 1. data_rho - loads density data for a single snapshot
# 2. data_var - loads all variables for a single snapshot
# 3. data_mul - loads all variables for multiple snapshots
data_rho = load_snapshot(Snap_meta, load_pic, llc, urc, "rho", use_level=level);
data_var = load_snapshot(Snap_meta, load_pic, llc, urc, use_level=level);
data_mul, times = load_multiple_snapshots(data_folder, 
                                      i_start, i_end, stride, 
                                      llc, urc,
                                      load_pic; 
                                      use_level=level)
println("Size of data_rho : ", size(data_rho))
println("Size of data_var : ", size(data_var))
println("Size of data_mul : ", size(data_mul))

In [ ]:
drop_dims = false
x,y,z,ds = get_xyz(Snap_meta, drop_dims, level; llc=llc, urc=urc); # for a subdomain defined by lower left corner (llc) and upper right corner (urc)
x_vals = x[:,1, 1]   # 1D arrays for interpolation
y_vals = y[1, :, 1]  # 1D arrays for interpolation
z_vals = z[1, 1, :]; # 1D arrays for interpolation

In [ ]:
#---------- interpolate data along the line ------------------------
line_interp_rho = interpolate_to_line(data_rho, x_vals, y_vals, z_vals, line);
line_interp_var = interpolate_to_line(data_var, x_vals, y_vals, z_vals, line);
line_interp_mul = interpolate_to_line(data_mul, x_vals, y_vals, z_vals, line);
println("Size of line_interp_rho: ", size(line_interp_rho))
println("Size of line_interp_var: ", size(line_interp_var))
println("Size of line_interp_mul: ", size(line_interp_mul))

In [ ]:
plane_interp_rho = interpolate_to_plane(data_rho, x_vals, y_vals, z_vals, grid);
plane_interp_var = interpolate_to_plane(data_var, x_vals, y_vals, z_vals, grid);
plane_interp_mul = interpolate_to_plane(data_mul, x_vals, y_vals, z_vals, grid);
println("Size of plane_interp_rho: ", size(plane_interp_rho))
println("Size of plane_interp_var: ", size(plane_interp_var))
println("Size of plane_interp_mul: ", size(plane_interp_mul))


In [ ]:
#------------------ convert momentum to velocity ------------------------
# For 2D, 3D, and 4D arrays, assumes that variables are in the last dimension
# For 5D arrays, assumes that variables are in second to last dimension
# var_idx is optional and can be used if variable dimension must be specified
mom_to_vel!(plane_interp_var, IDX);
mom_to_vel!(plane_interp_mul, IDX; var_idx=3);

In [ ]:
#------------- rotate vectors to be aligned with the plane --------------
#  the default is that 
#            - parallel component replaces "x" component
#            - perpendicular component replaces "y" component
#            - normal component replaces "z" component
# This can be overwritten by specifying `component_order`
# in this case, since the plane is xz it would make sense to have the normal 
#     component in the y direction, since y IS normal to the plane
#     e.i. we leave y component as it is and only rotate x and z components

# NOTE that this is done in-place, so the original data is modified

# default component order is (:par, :norm, :perp)
#rotate_aligned_vectors!(plane_interp_var, û, n̂, IDX)

# call with custom component order
rotate_aligned_vectors!(plane_interp_var, û, n̂, IDX; 
                component_order = (:par, :norm, :perp))


In [ ]:
#  the call assumes variable indices are in last dimension for 2D and 3D arrays
#  for hihger dimensions it assumes that the variable indices are in the second to last dimension
#  variable index can also be specified with `var_idx` argument

#rotate_aligned_vectors!(data_var, û, n̂, IDX; 
#            component_order = (:par, :norm, :perp),
#            var_idx = 4)

# Possible future extension

- generate_line_box (3D box with new coordinates)
